In [1]:
import requests
from bs4 import BeautifulSoup

# Get the main page content
url = "https://danskejernbaner.dk/vis.stations.oversigt.php"
main_url = "https://danskejernbaner.dk/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find the table-responsive div element
div_element = soup.find("div", class_="table-responsive")

# Find all the links within the div element
links = div_element.find_all("a", href=True)

# Create a dictionary to store station name and GPS coordinates
station_data = {}

# Loop through the links and scrape each subsite
for link in links:
    # Get the full url of the subsite
    subsite_url = main_url + link["href"]
    # Make a request to the subsite
    subsite_response = requests.get(subsite_url)
    # Parse the subsite content with BeautifulSoup
    subsite_soup = BeautifulSoup(subsite_response.text, "html.parser")
    
    # Find the station name
    article_element = subsite_soup.find("article", class_="text-justify-wide")
    if article_element:
        station_name_element = article_element.find("h1")
        if station_name_element:
            station_name = station_name_element.text.strip()
            # Find the GPS koordinater from the table
            table = subsite_soup.find("table", class_="table-condensed")
            rows = table.find_all("tr")
            for row in rows:
                columns = row.find_all("td")
                if columns[0].text.strip() == "GPS koordinater":
                    gps_coordinates = columns[1].text.strip()
                    # Add station name and GPS coordinates to the dictionary
                    station_data[station_name] = gps_coordinates
                    break
            else:
                print("GPS koordinater not found for", station_name)
        else:
            print("Station name element not found for", subsite_url)
    else:
        print("Article element not found for", subsite_url)
    break